In [1]:
#### 패키지 임포트
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import os
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
pd.set_option('display.max_columns', None)

In [2]:
#### 본문to형태소(morphs) 파일
df = pd.read_csv("../dataset/2022/본문_형태소분석/나무문화_본문_12월_형태소분석.txt", encoding='UTF-8')
df['morphs']=df['morphs'].apply(lambda x : x.strip("'][").split("', '"))
print(df.shape, df.columns)
docs = df['morphs'].to_list()
print(len(docs))

(1011113, 4) Index(['split_str', 'org_idx', 'morphs', 'pos_tag'], dtype='object')
1011113


In [3]:
df

,split_str,org_idx,morphs,pos_tag
0,흑흑 벌써 1년이 지났다니ㅠㅠ 주화의 2022년 요약 시작 ㅎㅅㅎ 2022 01 어...,0,"[흑, 흑, 벌써, 1년, 이, 지났다니, ㅠㅠ, 주화, 의, 2022년, 요약, ...","[('흑', 'Adverb'), ('흑', 'Adverb'), ('벌써', 'Nou..."
1,웃겨서 찍음,0,"[웃겨서, 찍음]","[('웃기다', 'Verb'), ('찍다', 'Verb')]"
2,그리고 김시연때뭉네 새해부터 부산감 광안리도 갔는데 횡단보도 기다리는 강아쥐 넘 귀...,0,"[그리고, 김시연, 때뭉, 네, 새해, 부터, 부산, 감, 광안, 리도, 갔는데, ...","[('그리고', 'Conjunction'), ('김시연', 'Noun'), ('때뭉..."
3,모범 강아쥐 그리고 바로 담날 최성빈이 경주오겠다고 놀아달라고 놀아달라고 해서 놀아줌ㅋ,0,"[모범, 강, 아, 쥐, 그리고, 바로, 담날, 최성빈, 이, 경주, 오겠다고, 놀...","[('모범', 'Noun'), ('강', 'Noun'), ('아', 'Josa'),..."
4,녹차라떼 거하게 쏟을 뻔 한거 최성빈이 잡아서 다행히 저만큼만 쏟음,0,"[녹차, 라떼, 거, 하게, 쏟을, 뻔, 한, 거, 최성빈, 이, 잡아서, 다행히,...","[('녹차', 'Noun'), ('라떼', 'Noun'), ('거', 'Noun')..."
...,...,...,...,...
1011108,돼지의 모습이 신기한지 우리 아이들 이곳에서 한참을 봤네요 ㅎㅎㅎ,20247,"[돼지, 의, 모습, 이, 신기한지, 우리, 아이, 들, 이, 곳, 에서, 한참, ...","[('돼지', 'Noun'), ('의', 'Josa'), ('모습', 'Noun')..."
1011109,남문 주변에는 닭 말 개 등 동물도 가까이서 볼 수 있어요 마을내에는 전통 민박에서...,20247,"[남문, 주변, 에는, 닭, 말, 개, 등, 동물, 도, 가까이, 서, 볼, 수, ...","[('남문', 'Noun'), ('주변', 'Noun'), ('에는', 'Josa'..."
1011110,문화유산과 공존하며 살아가는 마을 주민들의 삶을 느껴볼 수도 있다고 해요,20247,"[문화유산, 과, 공존, 하며, 살아가는, 마을, 주민, 들, 의, 삶, 을, 느껴...","[('문화유산', 'Noun'), ('과', 'Josa'), ('공존', 'Noun..."
1011111,제주 성읍민속마을 맛집 성읍치킨튀에서 맛있게 먹고 마을투어 해보는 것을 추천해요 옛...,20247,"[제주, 성, 읍민, 속, 마을, 맛집, 성, 읍, 치킨, 튀에서, 맛있게, 먹고,...","[('제주', 'Noun'), ('성', 'Modifier'), ('읍민', 'No..."


In [4]:
#### 각각의 본문이 str타입('나는 어제 밥을 먹었다')일 경우 list(['나', '는', '어제', '밥', '을' '먹다', '이다'])로 바꿔줌
docs2 = docs.copy()
# for i in tqdm(range(len(docs))):
#     docs2[i] = docs2[i].split()

In [5]:
#### 명사 빈도 상위 50개 단어 파일
topwords_df = pd.read_csv("../dataset/2022/본문_형태소분석/12월/나무문화_12월_형태소분석(명사).csv", encoding='UTF-8')
print(topwords_df.shape, topwords_df.columns)
topwords50 = topwords_df['명사'].to_list()[:50]
print(len(topwords50))
print(topwords50)

(164434, 2) Index(['명사', '명사빈도'], dtype='object')
50
['것', '수', '그', '이', '사람', '곳', '등', '때', '카페', '말', '우리', '더', '생각', '나', '문화', '내', '길', '위', '시간', '의', '나무', '층', '디저트', '사진', '중', '제', '전', '때문', '안', '일', '여행', '하나', '점', '로', '및', '위해', '한국', '마을', '집', '작품', '날', '대한', '시작', '개', '를', '저', '다른', '마음', '가장', '아이']


In [6]:
#### 중복 단어 제거하여 리스트 생성
unique_lines = [list(set(line)) for line in tqdm(docs2)]

100%|██████████| 1011113/1011113 [00:08<00:00, 125847.08it/s]


In [7]:
#### 중복 제거 잘 됐나 개수로 확인
print(len(docs[0]))
print(len(unique_lines[0]))

31
28


In [8]:
#### 전체 단어쌍 빈도 dict 형식으로 생성
freq_count = {}   #동시출현 빈도가 저장될 dict
for words in tqdm(unique_lines):
    # 전체 단어에 대한 동시 출현 빈도
    for i, a in enumerate(words):
        for b in words[i+1:]:
            if a == b: continue
            elif a>b:
                freq_count[b, a] = freq_count.get((b, a),0) + 1
            else :
                freq_count[a, b] = freq_count.get((a, b),0) + 1

100%|██████████| 1011113/1011113 [13:33<00:00, 1243.02it/s]


In [9]:
#### 딕셔너리를 데이터프레임에 담기
tt_freq_df=pd.DataFrame.from_dict(freq_count, orient='index')

In [10]:
#### 딕트to데이터프레임 저장
tt_freq_df.to_csv("../dataset/2022/단어동시출현행렬/12월/단어동시출현_딕셔너리_12월.csv", encoding='UTF-8')

In [11]:
#### dict형식을 컬럼명 지정해서 데이터프레임으로 만들기
list1 = [(tt_freq_df.index[i][0], tt_freq_df.index[i][1], tt_freq_df[0][i]) for i in tqdm(range(len(tt_freq_df)))]
freq_df2 = pd.DataFrame(list1, columns=["term1","term2","freq"])
print(freq_df2.shape)
freq_df2.head()

100%|██████████| 196958798/196958798 [16:32<00:00, 198353.19it/s]


(196958798, 3)


,term1,term2,freq
0,따,흑,25
1,벌써,흑,2
2,같지,흑,1
3,의,흑,477
4,후드,흑,1


In [12]:
#### 컬럼명 지정한 dict_to_데이터프레임 저장
freq_df2.to_csv("../dataset/2022/단어동시출현행렬/12월/단어동시출현_df_12월.csv", index=False, encoding='UTF-8')

In [13]:
#### 50위 이내에 속한 단어들로 이루어진 Dataframe을 추출
list2 =[(freq_df2.loc[i][0], freq_df2.loc[i][1], freq_df2.loc[i][2]) for i in tqdm(range(len(freq_df2))) if (freq_df2['term1'][i] in topwords50) &(freq_df2['term2'][i] in topwords50)]
freq_df3 = pd.DataFrame(list2, columns=["term1","term2","freq"])
freq_df3

100%|██████████| 196958798/196958798 [36:30<00:00, 89910.97it/s]


,term1,term2,freq
0,의,저,9749
1,의,이,214122
2,시작,의,12259
3,이,저,15405
4,시작,저,705
...,...,...,...
1220,다른,디저트,59
1221,디저트,말,73
1222,디저트,전,229
1223,디저트,점,547


In [14]:
#### 상위50위 단어들로 이루어진 단어쌍 데이터프레임 저장
freq_df3.to_csv("../dataset/2022/단어동시출현행렬/12월/단어동시출현_df_상위50_12월.csv", index=False, encoding='UTF-8')

In [15]:
#### 공출현빈도(1-mode matrix) 만들기
word_co_matx = np.zeros((50,50))
mat_idx = topwords50 # 단어동시출현행렬이니까 인덱스와 칼럼을 동일하게 지정
mat_col = topwords50
word_co_matx = pd.DataFrame(word_co_matx, index=mat_idx, columns=mat_col)
print(word_co_matx.shape)


#### 단어쌍의 첫 번째 단어가 인덱스와 같다 and 두 번째 단어가 칼럼과 같다 → 단어쌍빈도를 해당 셀 값에 삽입
for n in tqdm(range(len(freq_df3))):
    for idx in mat_idx:
        if idx == freq_df3['term1'][n]:
            for col in mat_col:
                if col == freq_df3['term2'][n]:
                    word_co_matx.loc[idx, col] = freq_df3['freq'][n]
                    word_co_matx.loc[col, idx] = freq_df3['freq'][n]

(50, 50)


100%|██████████| 1225/1225 [00:00<00:00, 1808.96it/s]


In [16]:
#### 1-mode 매트릭스 확인하기
word_co_matx

,것,수,그,이,사람,곳,등,때,카페,말,우리,더,생각,나,문화,내,길,위,시간,의,나무,층,디저트,사진,중,제,전,때문,안,일,여행,하나,점,로,및,위해,한국,마을,집,작품,날,대한,시작,개,를,저,다른,마음,가장,아이
것,0.0,30392.0,29130.0,112693.0,17747.0,9460.0,9133.0,14331.0,1535.0,14975.0,13216.0,10580.0,12778.0,15560.0,5936.0,11990.0,4271.0,8555.0,6647.0,85253.0,5406.0,1722.0,255.0,3922.0,7515.0,5766.0,7158.0,9575.0,8200.0,9915.0,3371.0,6452.0,3942.0,32786.0,2657.0,6232.0,3680.0,2073.0,4132.0,3859.0,4036.0,7013.0,4901.0,3350.0,65045.0,5747.0,7070.0,6027.0,6279.0,3770.0
수,30392.0,0.0,16994.0,70425.0,10950.0,11668.0,11214.0,8777.0,2534.0,8009.0,8244.0,8063.0,7265.0,10575.0,7051.0,7838.0,4692.0,6287.0,6841.0,62930.0,5338.0,2946.0,562.0,4198.0,5833.0,5372.0,6299.0,6587.0,5629.0,6658.0,3690.0,4209.0,3666.0,28678.0,3883.0,4162.0,3199.0,3033.0,3234.0,4066.0,3023.0,4752.0,3174.0,3732.0,54543.0,3499.0,4715.0,4040.0,4105.0,4092.0
그,29130.0,16994.0,0.0,62029.0,12378.0,8247.0,5905.0,10080.0,1075.0,9209.0,7619.0,5627.0,6439.0,9640.0,3802.0,7435.0,3268.0,5946.0,3582.0,60676.0,3571.0,1215.0,283.0,2276.0,6611.0,4071.0,4993.0,5544.0,5323.0,6045.0,1767.0,4073.0,2464.0,23124.0,1758.0,4054.0,2196.0,1721.0,3194.0,3332.0,2963.0,4340.0,3373.0,2384.0,44699.0,3088.0,3850.0,3774.0,3479.0,1907.0
이,112693.0,70425.0,62029.0,0.0,44258.0,42466.0,31576.0,36211.0,7300.0,29254.0,26185.0,24037.0,26150.0,35184.0,17895.0,26788.0,14540.0,19976.0,20819.0,214122.0,17263.0,7961.0,1128.0,15079.0,20396.0,17130.0,21876.0,19967.0,21680.0,20576.0,10371.0,14661.0,10395.0,91981.0,9502.0,14170.0,11288.0,9860.0,12976.0,12120.0,15478.0,13700.0,13988.0,11970.0,158503.0,15405.0,14501.0,14081.0,13394.0,11313.0
사람,17747.0,10950.0,12378.0,44258.0,0.0,4835.0,2894.0,5926.0,831.0,6099.0,4708.0,3878.0,5274.0,7033.0,2451.0,4594.0,2099.0,2890.0,2489.0,31125.0,2308.0,688.0,105.0,2171.0,2989.0,1911.0,2625.0,3431.0,3708.0,4094.0,1269.0,2408.0,1183.0,12457.0,593.0,2688.0,1388.0,1438.0,2053.0,1218.0,1956.0,2085.0,1791.0,1363.0,24855.0,2341.0,3947.0,2891.0,2063.0,1081.0
곳,9460.0,11668.0,8247.0,42466.0,4835.0,0.0,4226.0,4080.0,2320.0,2254.0,2815.0,3135.0,2839.0,4353.0,3610.0,3338.0,3470.0,2593.0,3289.0,25550.0,2797.0,1605.0,214.0,2984.0,3608.0,2661.0,2940.0,2021.0,2954.0,2133.0,3887.0,1984.0,1219.0,13919.0,1039.0,1808.0,1415.0,2953.0,1980.0,984.0,1734.0,950.0,1575.0,1782.0,20312.0,2672.0,2472.0,1749.0,2669.0,2449.0
등,9133.0,11214.0,5905.0,31576.0,2894.0,4226.0,0.0,3220.0,1020.0,2625.0,1908.0,1947.0,1476.0,4345.0,4657.0,2921.0,1572.0,3812.0,2081.0,31974.0,2896.0,1233.0,170.0,1503.0,3174.0,3449.0,4070.0,1678.0,1906.0,2511.0,1246.0,1312.0,1765.0,14327.0,4755.0,2382.0,2624.0,1518.0,1440.0,2113.0,1116.0,2466.0,1684.0,2785.0,22689.0,1144.0,1470.0,967.0,1514.0,1155.0
때,14331.0,8777.0,10080.0,36211.0,5926.0,4080.0,3220.0,0.0,695.0,4481.0,4089.0,3404.0,4175.0,5948.0,1781.0,4546.0,1846.0,2503.0,2515.0,25361.0,2040.0,603.0,115.0,1784.0,2566.0,2546.0,2810.0,2503.0,3377.0,3490.0,1611.0,2081.0,1132.0,12437.0,678.0,1891.0,1239.0,920.0,2071.0,1032.0,2105.0,1558.0,2193.0,1298.0,23097.0,2246.0,2125.0,2154.0,1925.0,1556.0
카페,1535.0,2534.0,1075.0,7300.0,831.0,2320.0,1020.0,695.0,0.0,359.0,693.0,1221.0,595.0,1247.0,1274.0,1102.0,1497.0,587.0,1136.0,4349.0,785.0,1785.0,1010.0,960.0,801.0,888.0,961.0,294.0,1010.0,1047.0,954.0,449.0,1006.0,3934.0,435.0,252.0,382.0,803.0,784.0,246.0,695.0,130.0,307.0,491.0,4282.0,537.0,331.0,365.0,269.0,596.0
말,14975.0,8009.0,9209.0,29254.0,6099.0,2254.0,2625.0,4481.0,359.0,0.0,3764.0,2646.0,3308.0,5293.0,1597.0,4175.0,1196.0,2329.0,1431.0,24128.0,1469.0,352.0,73.0,830.0,2087.0,1882.0,2243.0,2183.0,2758.0,2897.0,752.0,1727.0,916.0,9090.0,741.0,1617.0,1022.0,610.0,1296.0,829.0,1324.0,1722.0,1303.0,1054.0,18742.0,1604.0,1803.0,1891.0,1365.0,1127.0


In [17]:
#### 공출현빈도 데이터프레임 저장하기~~
#### ※ 인덱스가 저장돼야 하므로 index 파라미터 지정하지 말 것.
word_co_matx.to_csv("../dataset/2022/단어동시출현행렬/12월/공출현빈도_df_12월.csv", encoding='UTF-8')